<a href="https://colab.research.google.com/github/ArtemKarDev/Kaggle/blob/main/EDA_Netflix_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Netflix - развивающийся популярный стриминговый сервис. 

Проведем разведочный анализ данных с использованием интерактивной библиотеки визулизации данных - Plotly. <br>
Построим рекомендательную систему

Визуальный анализ данных 
1. Анализ набора данных Netflix
2. Составление системы рекомендаций для пользователей (Content based filtering)

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Загрузил датасет с https://www.kaggle.com/datasets/shivamb/netflix-shows
к себе на google-диск 

In [ ]:
# загружаем датасет в ноутбус с моего google-диска
!wget 'https://drive.google.com/uc?id=1JscRWVxO3DyBBBBr21fpeRHeUcmN5yhL' -O netflix_titles.csv


--2022-07-26 13:32:02--  https://drive.google.com/uc?id=1JscRWVxO3DyBBBBr21fpeRHeUcmN5yhL
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.113, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-14-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sct6v3jrji3dp0hu122mrvrfrskj900g/1658842275000/00838955718561688027/*/1JscRWVxO3DyBBBBr21fpeRHeUcmN5yhL?uuid=8ec991d2-696e-4aa5-8d6f-2e025b72e397 [following]
--2022-07-26 13:32:03--  https://doc-08-14-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sct6v3jrji3dp0hu122mrvrfrskj900g/1658842275000/00838955718561688027/*/1JscRWVxO3DyBBBBr21fpeRHeUcmN5yhL?uuid=8ec991d2-696e-4aa5-8d6f-2e025b72e397
Resolving doc-08-14-docs.googleusercontent.com (doc-08-14-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-08-14

In [ ]:
netflix_overall = pd.read_csv('netflix_titles.csv')
netflix_overall.sample(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
4632,s4633,TV Show,The Good Cop,NaN,"Tony Danza, Josh Groban, Monica Barbaro, Isiah...",United States,"September 21, 2018",2018,TV-14,1 Season,"Crime TV Shows, TV Comedies, TV Dramas","When he's not solving murders, a pathologicall..."
139,s140,Movie,Do the Right Thing,Spike Lee,"Danny Aiello, Ossie Davis, Ruby Dee, Richard E...",United States,"September 1, 2021",1989,R,120 min,"Classic Movies, Comedies, Dramas","On a sweltering day in Brooklyn, simmering rac..."
5172,s5173,Movie,Bon Cop Bad Cop 2,Alain Desrochers,"Patrick Huard, Colm Feore, Marc Beaupré, Noam ...",Canada,"November 11, 2017",2017,TV-MA,126 min,"Action & Adventure, Comedies, International Mo...",Two veteran buddy cops are back for a second c...
2497,s2498,Movie,I Will Follow,Ava DuVernay,"Salli Richardson-Whitfield, Michole Briana Whi...",United States,"May 20, 2020",2010,TV-14,80 min,"Dramas, Independent Movies, Romantic Movies","Devastated after her aunt dies, Maye moves out..."
4151,s4152,Movie,Virsa,Pankaj Batra,"Arya Babbar, Kanwaljeet Singh, Gulshan Grover,...","India, Pakistan","January 30, 2019",2010,TV-14,127 min,"Dramas, International Movies, Music & Musicals",A man is disheartened when he visits his well-...
5067,s5068,Movie,Todd Glass: Act Happy,Scott Moran,Todd Glass,United States,"January 23, 2018",2018,TV-MA,63 min,Stand-Up Comedy,Try to keep up as comedian Todd Glass delivers...
3312,s3313,Movie,Oththa Seruppu Size 7,Parthiban,Parthiban,India,"November 4, 2019",2019,TV-MA,103 min,"Dramas, International Movies, Thrillers","Taken into custody, a murder suspect's theatri..."
2167,s2168,Movie,Malibu Rescue: The Next Wave,Savage Steve Holland,"Ricardo Hurtado, Breanna Yde, Jackie R. Jacobs...",United States,"August 4, 2020",2020,TV-G,71 min,"Children & Family Movies, Comedies","As summer returns to Malibu, Team Flounder tak..."
4689,s4690,TV Show,The Innocents,NaN,"Sorcha Groundsell, Percelle Ascott, Guy Pearce...",United Kingdom,"August 24, 2018",2018,TV-MA,1 Season,"British TV Shows, International TV Shows, Roma...",Runaway teen lovers June and Harry find themse...
3219,s3220,TV Show,Levius,NaN,"Nobunaga Shimazaki, Junichi Suwabe, Takahiro S...",Japan,"November 28, 2019",2019,TV-14,1 Season,"Anime Series, International TV Shows",Young Levius rises through the ranks in the br...


In [ ]:
netflix_overall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


наблюдаем наличие пропусков в некоторых колонках, но для разведки это не кретично.

In [ ]:
netflix_shows = netflix_overall[netflix_overall['type'] == 'TV Show'].copy()

netflix_movie = netflix_overall[netflix_overall['type'] == 'Movie'].copy()

In [ ]:
netflix_counts = netflix_overall['type'].value_counts()
fig = px.bar(netflix_counts, x="type", color='type', title="Counts", text_auto='.3s')
fig.update_layout(autosize=False, width=600, height=400)
fig.show()

Количество фильмов на Netflix больше чем сериалов более чем в 2 раза

### Длительность видеоконтента

In [ ]:
topdirs = pd.value_counts(netflix_overall['duration'])
topdirs.head(10)

1 Season     1793
2 Seasons     425
3 Seasons     199
90 min        152
94 min        146
97 min        146
93 min        146
91 min        144
95 min        137
96 min        130
Name: duration, dtype: int64

In [ ]:
from plotly import graph_objects as go

fig = go.Figure([ 
      go.Bar(x=topdirs.index, 
             y=topdirs.values, 
             text=topdirs.values, 
             marker_color='indianred'
             ) 
      ])

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

Длительность в таблице указана в текстовом формате. Чаще всего указывается длительность - 'Season', тоесть Сезон - для твшоу(сериалов). но мы же знаем что фильмов больше, да - но длительность фильмов очень сильно вариативна.

# Анализ фильмов

## Какой месяц, самый перспективный для релиза контента?
вычислим месяц с наименьшим количеством публикуемого контента - для наименьшей конкуренции за зрителя

In [ ]:
# проверим есть ли пропуски в датах публикации фильма
netflix_overall.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [ ]:
# дропнем записи с пустой датой публикации
netflix_date = netflix_shows[['date_added']].dropna()
netflix_date.isna().sum()

date_added    0
dtype: int64

In [ ]:
# посмотрим что получилось
netflix_date.sample(5)

,date_added
1205,"March 15, 2021"
4976,"March 20, 2018"
1084,"April 9, 2021"
153,"September 1, 2021"
902,"May 9, 2021"


из этой записи необходимо выделить год и месяц - добавим их в новые столбцы

In [ ]:
netflix_date['year'] = netflix_date['date_added'].apply(lambda x: x.split(', ')[-1])
netflix_date['month'] = netflix_date['date_added'].apply(lambda x: x.lstrip().split(' ')[0])
netflix_date.head()

,date_added,year,month
1,"September 24, 2021",2021,September
2,"September 24, 2021",2021,September
3,"September 24, 2021",2021,September
4,"September 24, 2021",2021,September
5,"September 24, 2021",2021,September


In [ ]:
# посмотрим на самые ранние добавления фильмов
netflix_date.sort_values(by=['year','month']).head()

,date_added,year,month
6611,"February 4, 2008",2008,February
5940,"August 2, 2013",2013,August
7112,"March 31, 2013",2013,March
6885,"October 8, 2013",2013,October
7908,"October 14, 2013",2013,October


Интересно - между первой и второй публикацией фильмов 5 лет - видимо в этот период пуликовали только сериалы.

In [ ]:
# для сортировки по месяцу сделаем споком месяцев
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'] #[::-1]
df = netflix_date.groupby('year')['month'].value_counts().unstack().fillna(0)[month_order].T
df

year,2008,2013,2014,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,,,,
January,0.0,0.0,0.0,0.0,28.0,14.0,24.0,37.0,53.0,36.0
February,1.0,0.0,1.0,1.0,6.0,17.0,24.0,45.0,42.0,44.0
March,0.0,1.0,0.0,2.0,3.0,38.0,35.0,53.0,44.0,37.0
April,0.0,0.0,1.0,4.0,8.0,27.0,28.0,43.0,50.0,53.0
May,0.0,0.0,0.0,1.0,4.0,23.0,27.0,48.0,52.0,38.0
June,0.0,0.0,0.0,2.0,7.0,29.0,28.0,46.0,41.0,83.0
July,0.0,0.0,0.0,2.0,9.0,34.0,27.0,59.0,43.0,88.0
August,0.0,1.0,0.0,0.0,11.0,38.0,34.0,44.0,47.0,61.0
September,0.0,1.0,0.0,1.0,19.0,32.0,43.0,37.0,53.0,65.0


In [ ]:
fig = px.imshow(df,width=500, height=500)
fig.show()

С помощью plotly  очень просто визуализировать. Всё параметры применилиь автоматически - видим по самой яркой точке - июль 2021го самый "жаркий" месяц по публикациям фильмов.<br>
Но нам наиболее интересен наоборот самый холодный месяц - например январь несколько лет подряд в этот месяц наименьшее количество публикаций, но в связи с праздниками и длинными ввыходнымидолжно быть наибольшее количество просмотров. Но в США с 3го января уже начинают работать. Будем целиться на Российский сегмент зрителей.

## Рейтинги фильмов


исследуем взрасной рейтинг фильмов

In [ ]:
fig = go.Figure([ 
      go.Bar(y=netflix_movie['rating'].value_counts(), 
             x=netflix_movie['rating'].value_counts().index[:10], 
             text=netflix_movie['rating'].value_counts().values
             ) 
      ])

fig.update_layout(#legend_orientation="h",
                  #legend=dict(x=.5, xanchor="center"),
                  title="Количество фильмов по рейтингам",
                  xaxis_title="Рейтинги",
                  yaxis_title="Количество фильмов",
                  #margin=dict(l=0, r=0, t=30, b=0),
                  width=600, height=400)

fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.show()

1. ТV-МА - Самая большая группа. Такой рейтинг дается фильмам только для взрослой аудитории.
2. ТV-14 - Вторая группа по величине. Контент фильма с таким рейтингом не подходит для детей младше 14 лет.
3. R - Фильмы не подходят для просмотра детям до 17 лет.

## Анализ года выпуска фильма

In [ ]:
df_release = netflix_movie['release_year'].value_counts()[:15]

In [ ]:
fig = px.bar(y=df_release.values, 
             x=df_release.index, 
             text=df_release.values)
fig.update_layout(title="Количество фильмов в год",
                  xaxis_title="Год",
                  yaxis_title="Количество фильмов",
                  width=800, height=400)
fig.show()

2017 и 2018 самый частый год выпуска фильмов

## Топ 10 стран по выпуску фильмов

In [ ]:
#  проверим на пропуски 
netflix_movie.isna().sum()

show_id           0
type              0
title             0
director        188
cast            475
country         440
date_added        0
release_year      0
rating            2
duration          3
listed_in         0
description       0
dtype: int64

в колонке "страна" - есть пропуски  - но  не будем избовляться от этих записей - заполним их неизвестностью

In [ ]:
netflix_movie['country'].fillna("Unknow", inplace=True)
netflix_movie.isna().sum()

show_id           0
type              0
title             0
director        188
cast            475
country           0
date_added        0
release_year      0
rating            2
duration          3
listed_in         0
description       0
dtype: int64

In [ ]:
netflix_movie['country'].value_counts()

United States                            2058
India                                     893
Unknow                                    440
United Kingdom                            206
Canada                                    122
                                         ... 
United Kingdom, Russia, United States       1
Paraguay, Argentina                         1
United Kingdom, Malawi                      1
Austria, Iraq, United States                1
United Arab Emirates, Jordan                1
Name: country, Length: 652, dtype: int64

В некоторых записях есть перечень стран учавствовавших в создании фильма - 

In [ ]:
countries_mov = {} # подготовим словарь для хранения страны с количеством упонинаий в колонке country

cou = list(netflix_movie['country']) # соберем все записи из колонки country в список
for i in cou:               # пройдемся по этому списку
    i = i.replace(' ', '') # уберем пробелы если есть
    i = list(i.split(',')) # и разделим по запятым

    for j in i:   # теперь пройдемся по разделенным записям
        if j in list(countries_mov.keys()):  # и будем добавлять или увеличивать счетчик при наличии страны в словаре
            countries_mov[j] += 1
        else:
            countries_mov[j] = 1

# отсортируем и возьмем 10 первых чаще упоминающихся стран
countries_fin = sorted(countries_mov.items(), key=lambda item: countries_mov[item[0]], reverse=True)[:10]
countries_fin

[('UnitedStates', 2752),
 ('India', 962),
 ('UnitedKingdom', 534),
 ('Unknow', 440),
 ('Canada', 319),
 ('France', 303),
 ('Germany', 182),
 ('Spain', 171),
 ('Japan', 119),
 ('China', 114)]

In [ ]:
# визуализируем полученные данные
names = [name for name, values in countries_fin]
values = [values for name, values in countries_fin]

fig = px.bar(x=names, 
             y=values, 
             text=names)
fig.update_layout(title="Количество фильмов по странам",
                  xaxis_title="Страна",
                  yaxis_title="Количество фильмов",
                  width=800, height=400)
fig.show()

## Анализ продолжительноси фильмов


In [ ]:
netflix_movie['duration'].value_counts().head()

90 min    152
94 min    146
93 min    146
97 min    146
91 min    144
Name: duration, dtype: int64

In [ ]:
netflix_movie['duration'].isna().sum()

3

In [ ]:
netflix_movie['duration'].fillna(netflix_movie['duration'].mode()).shape

(6131,)

In [ ]:
netflix_movie['duration'] = netflix_movie['duration'].fillna(netflix_movie['duration'].mode())
netflix_movie['duration'] = netflix_movie['duration'].dropna()
netflix_movie[netflix_movie['duration'].isna()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...


преобразуем тип значений для отрисовки частотности

In [ ]:
# удалим слово min
netflix_movie['duration'] = netflix_movie['duration'].str.replace(' min', '')
# оставшиеся цифры переведем в int
netflix_movie['duration'] = netflix_movie['duration'].astype(int)

TypeError: ignored

In [ ]:
fig = px.histogram(x=netflix_movie['duration'])
fig.show()

## Анализ жанров

In [ ]:
! git init

Reinitialized existing Git repository in /content/.git/


In [ ]:
! cd drive/MyDrive/Colab_Notebooks/DS/kaggle

In [ ]:
! ls

drive  sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! git init
! git branch -M main
! git remote add origin https://github.com/ArtemKarDev/Kaggle.git
! git push -u origin main

Reinitialized existing Git repository in /content/.git/
error: refname refs/heads/master not found
fatal: Branch rename failed
fatal: remote origin already exists.
error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/ArtemKarDev/Kaggle.git'


In [ ]:
! git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
